In [ ]:
from ast import literal_eval
import glob2
import pandas as pd
import numpy as np
import json
import csv
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
top = pd.read_csv('label_top.csv', encoding='utf-8')
top

In [ ]:
df_top = pd.get_dummies(data = top, columns = ['top_cate', 'top_len', 'top_mat'])
df_top

In [ ]:
df_top.columns

In [ ]:
from keras.preprocessing import image
from tqdm import tqdm

In [ ]:

train_image = []
for i in tqdm(range(0, 300)):
    img = image.load_img('C:/img/'+df_top['image_name'][i],target_size=(400,400,3))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)

In [ ]:
print(len(train_image))

In [ ]:
X = np.array(train_image[:300])
X.shape


In [ ]:
plt.imshow(X[1])

In [ ]:
y = np.array(df_top.drop(['image_id', 'image_name', 'image_height', 'image_width'], axis=1)[:300])
len(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(400,400,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(38, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=64)

In [ ]:
img = image.load_img('C:/img2/(23)IMG_1.jpg',target_size=(400,400,3))
img = image.img_to_array(img)
img = img/255
prob = model.predict(img.reshape(1,400,400,3))
top_3 = np.argsort(prob[0])[:-4:-1]
top_3.tolist()

In [ ]:
loss_and_metrics = model.evaluate(X_test, y_test, batch_size=64)
print('')
print('loss_and_metrics : ' + str(loss_and_metrics))

In [ ]:
from keras.models import load_model

model.save('model_top.h5')
print('saved')